## ex1

In [4]:
# set some inputs
x = -2; y = 5; z = -4

q = x + y
f = q * z

dfdq = z
dfsz = q

dqdx = 1.0 * dfdq
dqdy = 1.0 * dfdq

print(dqdx)
print(dqdy)

-4.0
-4.0


## ex2

In [7]:
import math

w = [2,-3,-3] # assume some random weights and data
x = [-1, -2]

# forward pass
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1 / (1 + math.exp(-dot))

# backward pass through the neuron (backpropagation)
ddot = f * ( 1 -f )
dw = [ x[0]*ddot , x[1]*ddot , 1.0*ddot ]
dx = [ w[0]*ddot , w[1]*ddot ]

print(dw)
print(dx)

[-0.19661193324148185, -0.3932238664829637, 0.19661193324148185]
[0.3932238664829637, -0.5898357997244456]


## Backprop in practice: Staged computation

In [12]:
# my code

x = 3 # example values
y = -4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden # done!                                 #(8)

# backpropagation
#8
dfdnum = invden
dfdinvden = num

#7
dfdden = dfdinvden * ( -1.0/(den**2))

#6
dfdsigx = dfdden
dfdxpysqr = dfdden

#5
dfdxpy = dfdxpysqr * 2 * xpy

#4
dfdx = dfdxpy * (1.0)
dfdy = dfdxpy * (1.0)


#3
dfdx += dfdsigx * sigx * (1 - sigx)

#2
dfdx += dfdnum * (1.0)
dfdsigy = dfdnum * (1.0)

#1
dfdy += dfdsigy * sigy * (1 - sigy )

print(dfdx)
print(dfdy)

2.0595697955721652
1.5922327514838093


In [13]:
# answer

x = 3 # example values
y = -4

# forward pass
sigy = 1.0 / (1 + math.exp(-y)) # sigmoid in numerator   #(1)
num = x + sigy # numerator                               #(2)
sigx = 1.0 / (1 + math.exp(-x)) # sigmoid in denominator #(3)
xpy = x + y                                              #(4)
xpysqr = xpy**2                                          #(5)
den = sigx + xpysqr # denominator                        #(6)
invden = 1.0 / den                                       #(7)
f = num * invden # done!

# backprop f = num * invden
dnum = invden # gradient on numerator                             #(8)
dinvden = num                                                     #(8)
# backprop invden = 1.0 / den 
dden = (-1.0 / (den**2)) * dinvden                                #(7)
# backprop den = sigx + xpysqr
dsigx = (1) * dden                                                #(6)
dxpysqr = (1) * dden                                              #(6)
# backprop xpysqr = xpy**2
dxpy = (2 * xpy) * dxpysqr                                        #(5)
# backprop xpy = x + y
dx = (1) * dxpy                                                   #(4)
dy = (1) * dxpy                                                   #(4)
# backprop sigx = 1.0 / (1 + math.exp(-x))
dx += ((1 - sigx) * sigx) * dsigx # Notice += !! See notes below  #(3)
# backprop num = x + sigy
dx += (1) * dnum                                                  #(2)
dsigy = (1) * dnum                                                #(2)
# backprop sigy = 1.0 / (1 + math.exp(-y))
dy += ((1 - sigy) * sigy) * dsigy                                 #(1)
# done! phew

print(dx)
print(dy)

2.0595697955721652
1.5922327514838093


## Gradients for vectorized operations

In [18]:
import numpy as np

# forward pass
W = np.random.randn(5,10)
X = np.random.randn(10,3)
D = W.dot(X)

In [19]:
D.shape

(5, 3)

In [20]:
# now suppose we had the gradient on D from above in the circuit
dD = np.random.randn(*D.shape) # same shape as D
dW = dD.dot(X.T) #.T gives the transpose of the matrix
dX = W.T.dot(dD)

In [21]:
dD

array([[ 0.00546042, -1.25876204, -0.55680962],
       [-0.30180074, -0.02729946,  0.63547412],
       [-0.76913236,  1.40790803,  0.44814242],
       [-0.07390195, -0.37608441,  0.00894737],
       [-0.9682816 , -0.57364478,  1.62787242]])

In [22]:
dW

array([[ 0.07498633,  0.05192576,  2.69297514,  0.21401437, -0.11818966,
         1.70897097, -2.3677516 ,  0.11926526,  0.94370785, -2.80972571],
       [ 0.02849226,  0.96060496, -0.64080611, -0.17810062,  0.30202596,
        -0.30708519,  1.48965255, -0.2422224 ,  0.207198  , -0.11809927],
       [-1.93624032,  1.1039572 , -2.57732338, -0.42628593,  0.54214182,
        -3.1253848 ,  2.56855313, -0.36456606, -0.64838554,  1.27062376],
       [ 0.04323141,  0.26483059,  0.6134878 ,  0.01620425,  0.04353272,
         0.42702485, -0.29270175, -0.02845917,  0.32914647, -0.84108125],
       [-0.0919851 ,  2.96893429, -0.77724245, -0.45961621,  0.88566705,
        -0.36583136,  3.43591895, -0.6898978 ,  1.02330092, -1.65546131]])

In [23]:
dX

array([[ 0.62496173, -0.60477063,  1.15539713],
       [-1.27174916,  1.76861987,  0.47916554],
       [ 0.65331897, -0.07210365, -0.93589615],
       [ 1.45802413, -0.10059963, -3.02744657],
       [-0.89205363,  3.94862427,  0.75145071],
       [ 1.14392879, -1.39343847, -2.94672801],
       [-0.46327857,  0.28888793,  0.29461728],
       [-0.09176692,  1.36202942, -1.21678311],
       [-0.68868329,  2.19111234,  0.82928915],
       [ 0.28989003, -1.88217366, -0.660025  ]])